# GPT-2 for Sentiment Analysis on IMDb movie reviews

## Table of Contents
1. [Introduction](##Introduction)
2. [Data exploration](##Data-Exploration)
3. [Zero Shot Classification](##Zero-shot-classification)

## Introduction

The [IMDb](https://ai.stanford.edu/~amaas/data/sentiment/) is a binary sentiment classification dataset consisting of 100k movie reviews(50k positive and 50k negative). The dataset is split into train and test containing 50k reviews each.

In this notebook, my goals are:
1. Understand and implement [GPT-2](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf). Run GPT-2 on the IMDb classification task.
2. Fine-tune GPT-2 for sentiment classification in under ~30 minutes on a 8GB Nvidia 1080 GTX (Faster if you have a better, newer GPU).
3. Understand how [LoRA](https://arxiv.org/abs/2106.09685) is implemented and use it to fine-tune GPT-2 for sentiment classification.

## Data-Exploration
Get a summary of the dataset. i.e
1. No of samples
2. No of positive / negative samples.
3. Length of the movie reviews



In [ ]:
import sys
from pathlib import Path
# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))
import pandas
import torch
from torch.utils.data import Dataset

from gpt_config import GPTConfig
from reviewsDataset import reviewsDataset
from eval import Eval
from eval_config import EvalConfig
from train import Trainer
from train_config import TrainConfig
from gpt_utils import start_recording, stop_recording

In [ ]:
# Dataset exploration

imdb_train = reviewsDataset("train",max_length=10000)
imdb_test = reviewsDataset("test",max_length=10000)


def format_data(dataset: Dataset) -> pandas.DataFrame:

    data = []
    for batch in dataset:
        data.append({"input_ids":len(batch["input_ids"]),
                    "label": batch["label"],
                    "filename": batch["fpath"]})
    
    return pandas.DataFrame(data)

train_data = format_data(imdb_train)
test_data = format_data(imdb_test)


*Summary statistics of the dataset*

In [ ]:
def summary(data: pandas.DataFrame) -> None:
    print(f"Number of reviews: {len(data)}")
    print(f"Positive Reviews: {data[data['label'] == 1]['label'].count()}")
    print(f"Negative Reviews: {data[data['label'] == 0]['label'].count()}")
    print(f"Max Review Length: {data['input_ids'].max()}\nMin Review Length: {data['input_ids'].min()}")
    print(f"Median Review Length: {data['input_ids'].median()}\nMean Review Length: {data['input_ids'].mean()}")

print("Train\n--------------")
summary(train_data)
print("Test\n---------------")
summary(test_data)

*Length of reviews (measured by the number of tokens)*

In [ ]:
from matplotlib import pyplot as plt
def plot_hist(title: str,df: pandas.DataFrame) -> None:
    plt.figure()
    plt.hist(df["input_ids"],bins=100)
    plt.xlabel("No of tokens")
    plt.ylabel("Count")
    plt.title(f"{title}")

plot_hist(title='Train Data', df=train_data) 
plot_hist(title="Test Data", df=test_data)   

In [ ]:
plot_hist(title="Positive Reviews Test",df=test_data[test_data['label']==1])
plot_hist(title="Negative Reviews Test",df=test_data[test_data['label']==0])

Run the test.py in `sentiment_classification` and write the results to a file

In [ ]:
def get_metrics_by_bin(results, bins,threshold=0.5):
    TP = len(results[(results["label"] >= threshold) & (results["prediction"] >= threshold)])
    FP = len(results[(results["label"] < threshold) & (results["prediction"] >= threshold)])
    TN = len(results[(results["label"] < threshold) & (results["prediction"] < threshold)])
    FN = len(results[(results["label"] > threshold) & (results["prediction"] < threshold)])
    
    print("Metrics")
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    print(f"Precision: {precision}\nRecall: {recall}\nAccuracy: {accuracy}")
    results["bin"] = pandas.cut(results['length'],bins)
    metrics_by_bin = results.groupby('bin').apply(lambda x: pandas.Series({"TP": ((x["label"] >= threshold) & (x["prediction"] >= threshold)).sum(),
                                                                            "FP":((x["label"] < threshold) & (x["prediction"] >= threshold)).sum(),
                                                                            "FN": ((x["label"] >= threshold) & (x["prediction"] < threshold)).sum(),
                                                                            "TN": ((x["label"] < threshold) & (x["prediction"] < threshold)).sum()}))

    metrics_by_bin["accuracy"] = (metrics_by_bin["TP"] + metrics_by_bin["TN"])/(metrics_by_bin["TP"] + metrics_by_bin["TN"]+ metrics_by_bin["FP"]+ metrics_by_bin["FN"])
    metrics_by_bin["precision"] = metrics_by_bin["TP"]/(metrics_by_bin["TP"] + metrics_by_bin["FP"])
    metrics_by_bin["recall"] = metrics_by_bin["TP"]/(metrics_by_bin["TP"] + metrics_by_bin["FN"])
    print("Metrics by bin")
    print(metrics_by_bin.to_markdown())
    return {"precision":precision,"recall":recall,"accuracy":accuracy}

## Zero Shot

### Predict the next word given the following prompt
 
Review: {review} Sentiment:

I calculate the probabilities of the word " Positive" and " Negative" and classify the review based on which probability is greater.

**Run evaluation for the zero shot approach**

In [ ]:
start_recording("gpu_memory_log.txt")
model_config = GPTConfig(block_size=256,use_lora=False)
eval_config = EvalConfig(results_path="zero_shot_256.txt",subset=True,batch_size=128)
test_set = reviewsDataset(split="test",max_length=model_config.block_size)
evaluator = Eval(test_set=test_set,eval_config=eval_config,model_config=model_config)
evaluator.evaluate()
stop_recording()


In [ ]:
res_file = pandas.read_csv("zero_shot_256.txt")
bins = range(0,1500,256)
get_metrics_by_bin(res_file,bins,threshold=0.5)

**Finetuning without LoRA**

In [ ]:
start_recording("without_lora.txt")
train_config = TrainConfig(out_dir="run/10_layer_freeze/",init_from="gpt2",checkpoint_name="finetune_no_lora.ckpt")
model_config = GPTConfig(use_lora=False,block_size=256,binary_classification_head=True)
rd = reviewsDataset(split="train",max_length=model_config.block_size)
train_set, val_set = torch.utils.data.random_split(rd,[0.85,0.15])
trainer = Trainer(train_set,val_set,train_config,model_config)
trainer.train()
stop_recording()

**Run eval using the fine-tuned model**

In [ ]:
model_config = GPTConfig(block_size=256,use_lora=False,load_from_checkpoint=True,binary_classification_head=True,checkpoint_path="run/10_layer_freeze/finetune_no_lora.ckpt")
eval_config = EvalConfig(results_path="finetuned_no_lora.txt",subset=False)
test_set = reviewsDataset(split="test",cache_dir="/home/varun/Downloads/aclImdb/",max_length=256)
evaluator = Eval(test_set=test_set,eval_config=eval_config,model_config=model_config)
evaluator.evaluate()

**Test the performance of the fine-tuned model**

In [ ]:
res_file = pandas.read_csv("finetuned_no_lora.txt")
bins = range(0,1500,256)
get_metrics_by_bin(res_file,bins,threshold=0.5)

**Run training using LoRA**

In [ ]:
start_recording("with_lora.txt")
train_config = TrainConfig(out_dir="run/10_layer_freeze_lora",
                            checkpoint_name="finetune_lora.ckpt",
                            init_from="gpt2")
model_config = GPTConfig(block_size=256,
                        use_lora=True,
                        r=8,
                        lora_layers=(10,11),
                        binary_classification_head=True)
rd = reviewsDataset(split="train",max_length=model_config.block_size,cache_dir="/home/varun/Downloads/aclImdb/")
train_set, val_set = torch.utils.data.random_split(rd,[0.85,0.15])
trainer = Trainer(train_set,val_set,train_config,model_config)
trainer.train()
stop_recording()

**Evaluate using the LoRA finetuned model**

In [ ]:

model_config = GPTConfig(load_from_checkpoint=True,checkpoint_path="run/10_layer_freeze_lora/finetune_lora.ckpt")
eval_config = EvalConfig(results_path="run/10_layer_freeze_lora/finetuned_lora.txt",batch_size=8,subset=False)
test_set = reviewsDataset(split="test",cache_dir="/home/varun/Downloads/aclImdb/",max_length=model_config.block_size)
evaluator = Eval(test_set=test_set,eval_config=eval_config,model_config=model_config)
evaluator.evaluate()

In [ ]:
res_file = pandas.read_csv("run/10_layer_freeze_lora/finetuned_lora.txt")
bins = range(0,1500,256)
results = get_metrics_by_bin(res_file,bins,threshold=0.5)


In [ ]:
import pandas as pd

# Load data
log_file = "with_lora.txt"
data = pd.read_csv(log_file, names=["timestamp", "memory"], sep=", ")
data["memory"] = data["memory"].str.replace(" MiB", "").astype(int)

# Plot memory usage
plt.figure(figsize=(10, 5))
plt.plot(pd.to_datetime(data["timestamp"]), data["memory"], label="GPU Memory Usage")
plt.xlabel("Time")
plt.ylabel("Memory Usage (MB)")
plt.title("GPU Memory Usage Over Time")
plt.legend()
plt.grid(True)
plt.show()


In [1]:
import torch
from train_bert import Trainer
from reviewsDataset import reviewsDataset
from language_models.bert_config import BERTTrainConfig

train_set, val_set = torch.utils.data.random_split(reviewsDataset("train"),[0.9,0.1])
test_set = reviewsDataset("test") 
config = BERTTrainConfig(model_type="sc-bert",freeze_layers=10,checkpoint_name="sc_bert_lora.pt",eval_size=50,init_from="bert",max_iters=14000)
criterion = torch.nn.BCEWithLogitsLoss(reduction="sum")
trainer = Trainer(config,train_set,val_set,test_set,criterion)

Loading pre-trained weights for bert
Setting up LoRA for layer 10
Setting up LoRA for layer 11


In [2]:
trainer.train()

Training..


  0%|          | 1/14000 [00:28<110:18:23, 28.37s/it]

Step: 0
Train Loss: 0.7058370113372803
Validation Loss:0.6977845430374146
Train Error: 0.52
Validation Error: 0.52
Test Error: 0.54


 11%|█         | 1561/14000 [04:15<30:17,  6.85it/s] 

In [6]:
import torch
from train_bert import Trainer
from reviewsDataset import reviewsDataset
from language_models.bert_config import BERTTrainConfig


train_set, val_set = torch.utils.data.random_split(reviewsDataset("train"),[0.9,0.1])
test_set = reviewsDataset(split="test")
config = BERTTrainConfig(model_type="sc-bert",init_from="resume",freeze_layers=10,checkpoint_name="sc_bert_lora.pt",eval_size=50)
criterion = torch.nn.BCEWithLogitsLoss(reduction="sum")
trainer = Trainer(config,train_set,val_set,test_set,criterion)
print(trainer.calculate_subset_error(test_set))

Loading model from out/sc_bert.pt


100%|██████████| 3125/3125 [06:58<00:00,  7.46it/s]

0.10719999999999996
